In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from tqdm.notebook import tqdm
import time
import pdb
import csv
from PIL import Image
import requests
import asyncio
import os
import re

In [15]:
playwright = await async_playwright().start()
from playwright.async_api import ElementHandle
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

## EXTRACTING NAMES OF ALL COUNTRIES IN THE WORLD

In [2]:
country_names_url = "https://data.worldbank.org/country"

In [3]:
await page.goto(country_names_url)

<Response url='https://data.worldbank.org/country' request=<Request url='https://data.worldbank.org/country' method='GET'>>

In [7]:
world_country_list = await page.locator('.nav-item ul li').all_text_contents()
world_country_list

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'Fren

In [9]:
world_country_list

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'Fren

### Saving as CSV

In [10]:
pd.DataFrame(world_country_list).to_csv('world_countries_list.csv', index=False)

In [3]:
country_list = pd.read_csv('world_countries_list.csv')
country_list

,0
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra
...,...
212,Virgin Islands (U.S.)
213,West Bank and Gaza
214,"Yemen, Rep."
215,Zambia


### Cleaning

In [4]:
country_list.columns = country_list.columns.str.replace('0', "name")

In [5]:
country_list['name'] = country_list['name'].str.replace("[(]", "").str.replace("[)]", "").str.replace(" The", "")

/var/folders/6g/mf4qklbs63q07skjqqfkj5lw0000gn/T/ipykernel_74118/989574333.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  country_list['name'] = country_list['name'].str.replace("[(]", "").str.replace("[)]", "").str.replace(" The", "")


In [6]:
country_list[country_list['name'].str.contains("Korea")]

,name
103,"Korea, Dem. People's Rep."
104,"Korea, Rep."


**Renaming South and North Korea:**

In [7]:
country_list['name'] = country_list['name'].str.replace("Korea, Dem. People's Rep.", "North Korea").str.replace("Korea, Rep.", "South Korea")

/var/folders/6g/mf4qklbs63q07skjqqfkj5lw0000gn/T/ipykernel_74118/3075129092.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  country_list['name'] = country_list['name'].str.replace("Korea, Dem. People's Rep.", "North Korea").str.replace("Korea, Rep.", "South Korea")


In [8]:
#Sanity check
country_list[country_list['name'].str.contains("Rep")]

,name
37,Central African Republic
44,"Congo, Dem. Rep."
45,"Congo, Rep."
56,Dominican Republic
58,"Egypt, Arab Rep."
91,"Iran, Islamic Rep."
107,Kyrgyz Republic
173,Slovak Republic
189,Syrian Arab Republic
214,"Yemen, Rep."


**Renaming " Dem." and " Rep." with " Democractic" and " Republic":**

In [9]:
country_list['name'] = country_list['name'].str.replace(" Dem[.]", " Democratic").str.replace(" Rep[.]", " Republic")

/var/folders/6g/mf4qklbs63q07skjqqfkj5lw0000gn/T/ipykernel_74118/3153192959.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  country_list['name'] = country_list['name'].str.replace(" Dem[.]", " Democratic").str.replace(" Rep[.]", " Republic")


**Preparing country names for URL handling by replaceing spaces with "+":**

In [10]:
country_list['name'] = country_list['name'].str.replace(",", "").str.replace(" ", "+")

In [11]:
world_country_list_clean = list(country_list['name'])

In [13]:
country_list.to_csv('world_countries_list_semi-clean.csv')

,name
0,Afghanistan
1,Albania
2,Algeria
3,American+Samoa
4,Andorra
...,...
212,Virgin+Islands+U.S.
213,West+Bank+and+Gaza
214,Yemen+Republic
215,Zambia


## Scraping

In [34]:
url_list = []
country_wise_result_count = []
for country in world_country_list_clean:
    url = 'https://commons.wikimedia.org/w/index.php?search=%22'+country+'%22&title=Special:MediaSearch&type=video'
    country_dict = {}
    country_dict['country'] = country
    await page.goto(url)
    time.sleep(4.5)
    try:
        video_count_element = page.locator('#video-count')
        if video_count_element == "":
            print("Element with ID 'video-count' not found.")
            print(f"Scraper stopped after {len(url_list)} countries.")
            continue
        video_count = await video_count_element.text_content()     
        country_dict['count'] = video_count
        if country_dict in country_wise_result_count:
            continue
        url_list.append(url)
        country_wise_result_count.append(country_dict)
        print(f"Finished scraping {len(country_wise_result_count)} countries.")
    except Exception as e:
                print(f"An error occurred: {e}")

Finished scraping 1 countries.
Finished scraping 2 countries.
Finished scraping 3 countries.
Finished scraping 4 countries.
Finished scraping 5 countries.
Finished scraping 6 countries.
An error occurred: Timeout 30000ms exceeded.
=========================== logs ===========================
waiting for locator("#video-count")
Finished scraping 7 countries.
Finished scraping 8 countries.
Finished scraping 9 countries.
Finished scraping 10 countries.
Finished scraping 11 countries.
Finished scraping 12 countries.
Finished scraping 13 countries.
Finished scraping 14 countries.
Finished scraping 15 countries.
Finished scraping 16 countries.
Finished scraping 17 countries.
Finished scraping 18 countries.
Finished scraping 19 countries.
Finished scraping 20 countries.
Finished scraping 21 countries.
Finished scraping 22 countries.
Finished scraping 23 countries.
Finished scraping 24 countries.
Finished scraping 25 countries.
Finished scraping 26 countries.
An error occurred: Timeout 30000ms 

An error occurred: Timeout 30000ms exceeded.
=========================== logs ===========================
waiting for locator("#video-count")
Finished scraping 171 countries.
Finished scraping 172 countries.
Finished scraping 173 countries.
Finished scraping 174 countries.
Finished scraping 175 countries.
Finished scraping 176 countries.
Finished scraping 177 countries.
Finished scraping 178 countries.
Finished scraping 179 countries.
Finished scraping 180 countries.
Finished scraping 181 countries.
Finished scraping 182 countries.
Finished scraping 183 countries.
Finished scraping 184 countries.
Finished scraping 185 countries.
An error occurred: Timeout 30000ms exceeded.
=========================== logs ===========================
waiting for locator("#video-count")
Finished scraping 186 countries.
Finished scraping 187 countries.
Finished scraping 188 countries.
Finished scraping 189 countries.
Finished scraping 190 countries.
Finished scraping 191 countries.
Finished scraping 192 c

In [35]:
len(url_list)

198

In [ ]:
#add patch to account for zero result countries
#try to replace "try-replace" with ifelse
#add the while statement to create a csv + html in the scraper itself


In [37]:
df = pd.DataFrame(country_wise_result_count)
df

,country,count
0,Afghanistan,238 results
1,Albania,44 results
2,Algeria,86 results
3,American+Samoa,16 results
4,Andorra,10 results
...,...,...
193,Vanuatu,29 results
194,Viet+Nam,36 results
195,West+Bank+and+Gaza,1 result
196,Zambia,78 results


In [38]:
df['count'] = df['count'].str.replace(" results", "").str.replace(" result", "")

In [39]:
df['country'] = df['country'].str.replace("[+]", " ").str.replace(",", "")

/var/folders/6g/mf4qklbs63q07skjqqfkj5lw0000gn/T/ipykernel_71763/674604883.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['country'] = df['country'].str.replace("[+]", " ").str.replace(",", "")


In [40]:
df['count'] = df['count'].str.replace(",", "")

In [41]:
df

,country,count
0,Afghanistan,238
1,Albania,44
2,Algeria,86
3,American Samoa,16
4,Andorra,10
...,...,...
193,Vanuatu,29
194,Viet Nam,36
195,West Bank and Gaza,1
196,Zambia,78


In [42]:
df.to_csv('video_results.csv', index=False)

In [43]:
df.dtypes

country    object
count      object
dtype: object

In [44]:
df['count'] = df['count'].astype(int)

In [45]:
df.sort_values(by='count', ascending=False).reset_index(drop=True).head(10)

,country,count
0,United States,56543
1,Indonesia,29587
2,France,12476
3,Netherlands,12433
4,Mexico,8152
5,Germany,7626
6,Slovenia,6860
7,China,4948
8,Colombia,4747
9,Jamaica,4424


In [46]:
len(country_wise_result_count)

198

In [47]:
len(url_list)

198

In [ ]:
# with open(f"trial_2_court_page_htmls/{file_name}", 'a') as fp:
#     print(f"Saved as {file_name}")
#     fp.write(page_content + '<div class="dist_name">' + district_text + "</div>")